In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, StandardScaler
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

In [91]:
def load_data():
    df = pd.read_csv('../data/kzhan176/citizens_data/BrownDSI_masked_capstone_data.csv_20250401031515')
    return df

In [92]:
def process_calgorial_features(df):

    '''
    one hot encoding:
    - masked_bank_num
    - masked_account_type
    - masked_product_code_grouped
    - relationship_balance_new_account
    - oao_flg
    - onus_ind
    - treasury_check_ind
    - heloc_ind
    '''

    # drop unused columns
    cols_to_drop = ['masked_dep_acct_num', 'masked_id', 'channel', 'drawee_avg', 'drawee_max', 'drawee_min', 'RDI_DT', 'RETURN_REASON', 'over_draft_amount']
    df.drop(columns=cols_to_drop, inplace=True)

    # process masked_product_code
    top_n = 8
    top_product_codes = df['masked_product_code'].value_counts().nlargest(top_n).index
    df['masked_product_code_grouped'] = df['masked_product_code'].apply(
        lambda x: f'prod_{x}' if x in top_product_codes else 'Other'
    )
    df.drop(columns='masked_product_code', inplace=True)

    one_hot_features = ['masked_bank_num', 'masked_account_type', 'masked_product_code_grouped']
    df = pd.get_dummies(df, columns=one_hot_features)
    
    df['oao_flg'] = (df['oao_flg'] == 'Y').astype(int)

    cat_ftrs = ['onus_ind', 'treasury_check_ind', 'heloc_ind']
    categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant',fill_value='other')),
    ('onehot', OneHotEncoder(sparse_output=False,handle_unknown='ignore'))])

    '''
    ordinal encoding
    - bucket_days_since_open
    - deposit_quarter
    - deposit_dayofweek
    '''

    # fill missing values with -1
    df['bucket_days_since_open'] = df['bucket_days_since_open'].fillna(-1)
    ordinal_mapping = {
        '0-1000': 0,
        '1000-2000': 1,
        '2000-5000': 2,
        '5000+': 3
    }
    df['bucket_days_since_open'] = df['bucket_days_since_open'].map(ordinal_mapping)
    print(df['bucket_days_since_open'].unique())

    df['deposit_dt'] = pd.to_datetime(df['deposit_dt'])
    df['deposit_quarter'] = df['deposit_dt'].dt.quarter
    df['deposit_dayofweek'] = df['deposit_dt'].dt.dayofweek
    df.drop(columns='deposit_dt', inplace=True)

    return categorical_transformer, cat_ftrs


def process_continous_features(df):
    # total_deposit_amount
    df['total_deposit_amount'] = df['total_deposit_amount'].apply(lambda x: np.log1p(x) if x > 0 else 0)

    # item_amt
    df['item_amt'] = df['item_amt'].apply(lambda x: np.log1p(x) if x > 0 else 0)

    # relationship_balance
    df['relationship_balance_new_account'] = (df['relationship_balance'] == -99999999).astype(int)

    df['relationship_balance'] = df['relationship_balance'].replace(-99999999, np.nan)
    median_val = df['relationship_balance'].median()
    df['relationship_balance'] = df['relationship_balance'].fillna(median_val)
    min_val = df['relationship_balance'].min()
    shift = 1 - min_val if min_val <= 0 else 0
    df['relationship_balance'] = np.log1p(df['relationship_balance'] + shift)

    # rdis, max_deposit_amount30d, total_deposit_item_count
    # drawee_sum, drawee_cnt
    num_ftrs = ['drawee_sum', 'drawee_cnt']
    log_num_ftrs = ['rdis', 'max_deposit_amount30d', 'total_deposit_item_count']

    df['rdis'] = df['rdis'].fillna(0)
    df['drawee_sum'] = df['drawee_sum'].fillna(0)
    df['drawee_cnt'] = df['drawee_cnt'].fillna(0)

    numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

    log_numeric_transformer = Pipeline(steps=[
        ('log', FunctionTransformer(np.log1p, feature_names_out = 'one-to-one')),
        ('scaler', RobustScaler())])
    
    return log_numeric_transformer, log_num_ftrs, numeric_transformer, num_ftrs

'''
helper function for process_prevtran_features
'''
def prevtran_missing_imputation(df, flag="zero"):
    prevtran_cols = [f'prevtran{i}' for i in range(1, 11)]
    if flag == "zero":
        imputer = SimpleImputer(strategy='constant', fill_value=0)
        df[prevtran_cols] = imputer.fit_transform(df[prevtran_cols])
    else:
        raise ValueError(f"Invalid flag: {flag}. Only 'zero' is supported.")

'''
helper function for process_prevtran_features
'''
def prevtrandate_missing_imputation(df, flag="prevdate"):

    if flag == "prevdate":
        def impute_prevtrandate(row):
            # Find the last available value (starting from 10 to 1)
            for i in reversed(range(10)):
                val = row[prevtrandate_cols[i]]
                if not pd.isna(val):
                    return row.fillna(val)
            return row.fillna(0)

        prevtrandate_cols = [f'prevtrandate{i}' for i in range(1, 11)]
        df[prevtrandate_cols] = df[prevtrandate_cols].apply(impute_prevtrandate, axis=1)
    else:
        raise ValueError(f"Invalid flag: {flag}. Only 'prevdate' is supported.")

'''
Helper function for process_prevtran_features
'''
from statsmodels.tsa.stattools import acf

def compute_acf_lag1(row):
    x = row.values.astype(float)
    return pd.Series([acf(x, nlags=1, fft=False)[1]])

'''
deal with prevtran* and prevtrandate*
'''
def process_prevtran_features(df):

    prevtran_cols = [f'prevtran{i}' for i in range(1, 11)]
    keep_prevtran_cols = [f'prevtran{i}' for i in range(1, 7)]  # keep prevtran1–6
    drop_prevtran_cols = [f'prevtran{i}' for i in range(7, 11)]

    prevtrandate_cols = [f'prevtrandate{i}' for i in range(1, 11)]
    keep_prevtrandate_cols = [f'prevtrandate{i}' for i in range(1, 4)]  # keep prevtrandate1–3
    drop_prevtrandate_cols = [f'prevtrandate{i}' for i in range(4, 11)]

    # Mean, median, std, min, max, and sum across previous transactions, 
    # preserve absence of behavior in the statistics by computing before imputing
    df['prevtran_mean'] = df[prevtran_cols].mean(axis=1, skipna=True)
    df['prevtran_std'] = df[prevtran_cols].std(axis=1, skipna=True)
    df['prevtran_sum'] = df[prevtran_cols].sum(axis=1, skipna=True)
    df['prevtran_valid_count'] = df[prevtran_cols].notnull().sum(axis=1)

    # prevtrandate summary stats
    df['prevtrandate_mean'] = df[prevtrandate_cols].mean(axis=1, skipna=True)
    df['prevtrandate_std'] = df[prevtrandate_cols].std(axis=1, skipna=True)

    # add an isoutlier column to preserve extreme behavior as a signal
    threshold = df[prevtran_cols].stack().quantile(0.99)
    df['isoutlier_prevtran'] = (df[prevtran_cols].max(axis=1) > threshold).astype(int)

    # impute missing values
    prevtran_missing_imputation(df)
    prevtrandate_missing_imputation(df)
    # if not check_prevtran_imputed:
    #     raise ValueError(f"Imputation incomplete: Some prevtran/prevtrandate values are still missing.")

    # take log of prevtran1–6, since data right-skewed
    df[keep_prevtran_cols] = df[keep_prevtran_cols].applymap(lambda x: np.log1p(x) if x > 0 else 0)

    # prevtran_diff0, prevtran_ratio0
    df['prevtran_diff0'] = df['total_deposit_amount'] - df['prevtran1']
    df['prevtran_ratio0'] = df['total_deposit_amount'] / (df['prevtran1'] + 1e-3)
    
    # prevtran_diff1, prevtran_ratio1
    df['prevtran_diff1'] = df['prevtran1'] - df['prevtran2']
    df['prevtran_ratio1'] = df['prevtran1'] / (df['prevtran2'] + 1e-3)
    
    # timeseries features: acf
    df[['acf_lag1']] = df[keep_prevtran_cols].apply(compute_acf_lag1, axis=1)

    # rolling window features
    # TODO
    
    # drop columns
    columns_to_drop = (drop_prevtran_cols + drop_prevtrandate_cols)
    df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)


def normalization(df):
    robust_scaler = RobustScaler()
    standard_scaler = StandardScaler()

    keep_prevtran_cols = [f'prevtran{i}' for i in range(1, 7)]
    keep_prevtrandate_cols = [f'prevtrandate{i}' for i in range(1, 4)]

    robust_scalar_features = ['total_deposit_amount', 'item_amt'] + keep_prevtran_cols + \
                        ['prevtran_mean', 'prevtran_std', 'prevtran_sum', 'prevtran_diff0', 'prevtran_ratio0'] + \
                        ['prevtran_diff1', 'prevtran_ratio1', 'acf_lag1']
    standard_scaler_features = keep_prevtrandate_cols + ['relationship_balance', 'drawee_sum', 'drawee_cnt',
   'rdis', 'max_deposit_amount30d', 'total_deposit_item_count'] + ['prevtrandate_mean', 'prevtrandate_std']
    
    return robust_scaler, robust_scalar_features, standard_scaler, standard_scaler_features


def feature_engineer(df):

    categorical_transformer, cat_ftrs = process_calgorial_features(df)
    log_numeric_transformer, log_num_ftrs, numeric_transformer, num_ftrs = process_continous_features(df)
    process_prevtran_features(df)
    robust_scaler, robust_scalar_features, standard_scaler, standard_scaler_features = normalization(df)
    
    preprocessor = ColumnTransformer(
        transformers=[
            # From categorical_transformer
            ('cat', categorical_transformer, cat_ftrs),

            # From continous_transformer (log and standard scaled)
            ('log_num', log_numeric_transformer, log_num_ftrs),
            ('num', numeric_transformer, num_ftrs),

            # From normalizer
            ('robust_scaler', robust_scaler, robust_scalar_features),
            ('standard_scaler', standard_scaler, standard_scaler_features)
        ],
        remainder='passthrough')
    X_transformed = preprocessor.fit_transform(df)
    feature_names = preprocessor.get_feature_names_out()
    df_final = pd.DataFrame(X_transformed, columns=feature_names, index=df.index)
    
    return X_transformed, df_final

In [93]:
df = load_data()
X_transformed, df_final = feature_engineer(df)

[ 3.  0.  2.  1. nan]


/tmp/ipykernel_1618341/694642103.py:172: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[keep_prevtran_cols] = df[keep_prevtran_cols].applymap(lambda x: np.log1p(x) if x > 0 else 0)
/users/kzhan176/.conda/envs/citizens/lib/python3.12/site-packages/statsmodels/tsa/stattools.py:702: RuntimeWarning: invalid value encountered in divide
  acf = avf[: nlags + 1] / avf[0]


In [94]:
df_original = load_data()
df_original['bucket_days_since_open'].unique()

array(['5000+', '0-1000', '2000-5000', '1000-2000', nan], dtype=object)

In [95]:
df_final

,cat__onus_ind_F,cat__onus_ind_T,cat__treasury_check_ind_N,cat__treasury_check_ind_Y,cat__heloc_ind_N,cat__heloc_ind_Y,log_num__rdis,log_num__max_deposit_amount30d,log_num__total_deposit_item_count,num__drawee_sum,...,remainder__bucket_days_since_open,remainder__number_of_owners,remainder__deposit_dt,remainder__oao_flg,remainder__return_target,remainder__month_num,remainder__masked_product_code_grouped,remainder__relationship_balance_new_account,remainder__prevtran_valid_count,remainder__isoutlier_prevtran
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.090366,0.0,0.090942,...,5000+,1,2024-08-01,N,0,8,prod_0,0,10,0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-0.277019,0.0,-0.127813,...,0-1000,1,2024-04-25,N,0,4,prod_1,0,2,0
2,1.0,0.0,1.0,0.0,1.0,0.0,6.475733,-0.817824,1.0,-0.146756,...,5000+,0,2024-03-18,N,0,3,prod_0,0,1,0
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.346936,0.0,-0.146756,...,5000+,0,2024-11-29,N,0,11,prod_1,0,7,0
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.325634,2.709511,0.845855,...,2000-5000,2,2024-05-29,N,0,5,prod_1,0,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17760,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.182176,0.0,-0.146756,...,5000+,2,2024-11-05,N,1,11,prod_0,0,1,0
17761,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.27333,0.0,-0.146756,...,0-1000,1,2024-11-06,N,1,11,prod_1,0,9,0
17762,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-0.817824,1.0,-0.146756,...,5000+,0,2024-11-12,N,1,11,Other,0,10,0
17763,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-0.817824,0.0,-0.146756,...,5000+,0,2024-11-18,N,1,11,prod_1,0,10,0


In [96]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17765 entries, 0 to 17764
Data columns (total 50 columns):
 #   Column                                       Non-Null Count  Dtype 
---  ------                                       --------------  ----- 
 0   cat__onus_ind_F                              17765 non-null  object
 1   cat__onus_ind_T                              17765 non-null  object
 2   cat__treasury_check_ind_N                    17765 non-null  object
 3   cat__treasury_check_ind_Y                    17765 non-null  object
 4   cat__heloc_ind_N                             17765 non-null  object
 5   cat__heloc_ind_Y                             17765 non-null  object
 6   log_num__rdis                                17765 non-null  object
 7   log_num__max_deposit_amount30d               17765 non-null  object
 8   log_num__total_deposit_item_count            17765 non-null  object
 9   num__drawee_sum                              17765 non-null  object
 10  num__drawe